# Page rank

## Imports

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import pandas as pd

import networkx as nx
import graphviz as gv
import random

from typing import List

## Network classes

In [ ]:
class Page:
    """A class representing a webpage."""
    def __init__(self, name):
        """Constructor method, takes the name of the webpage as an argument."""
        self.name = name
        self.outgoingLinks = []
        self.incomingLinks = []
        
    def __repr__(self):
        """A page is represented by its name."""
        return self.name
    
    def addLinkFrom(self, other):
        """Adds a link from another page, ending at this page"""
        self.incomingLinks.append(other)
    
    def addLinkTo(self, other):
        """Adds a link to another page, starting from this page"""
        self.outgoingLinks.append(other)
        other.addLinkFrom(self)
        
    def addLinksTo(self, *args):
        """Adds a link to another page, starting from this page"""
        for page in args:
            self.addLinkTo(page)
        
    def outDegree(self):
        """Returns the amount of links this page has to other pages, i.e. its out-degree"""
        return len(self.outgoingLinks)
    
    def getName(self):
        return self.name
    
    def linksTo(self, other) -> bool:
        return other in self.outgoingLinks

In [ ]:
class Network:
    """A class representing a network of webpages"""
    def __init__(self):
        """A network starts without any pages."""
        self.pages = []
        self.counter = 0
        
        # Whether the dataframe is out of date
        self.old_df = True
    
    def __iter__(self):
        """Iterating over a network is the same as iterating over its pages."""
        return iter(self.pages)
        
    def addPage(self, page: Page):
        """Adds a page to the network"""
        # give the page an id (since the name of the webpage doesn't have to be unique)
        page.id = self.counter
        self.counter += 1
        self.pages.append(page)
        
        # Dataframe is out of date now
        self.old_df = True
        
    def addPages(self, *args):
        """Adds multiple pages to the network"""
        for page in args:
            self.addPage(page)
        
    def getPages(self):
        """Returns the pages in the network."""
        return self.pages
    
    def size(self):
        """Returns the size of the network."""
        return len(self.pages)
    
    def showRanking(self):
        """
        Prints the pages with their corresponding rank. 
        Not sorted on anything in particular, prints in increasing id.
        """
        for page in self:
            print(f'{page.name}: {page.rank}')
            
    def debugRanking(self):
        """
        Computes the sum of the rank of all pages in the network, which *should* return 1.
        """
        rankSum = 0
        for page in self:
            rankSum += page.rank
        print(rankSum)

    def getRankingOrder(self):
        """
        Returns a dictionary of pages with their corresponding order statistic (e.g. 1st, 2nd etc), sorted in increasing order.
        Note two pages may have the same order statistic
        """
        ranks = {page: page.rank for page in self}
        sorted_ranks = {k: v for k, v in sorted(ranks.items(), key=lambda item: item[1], reverse=True)}

        sorted_orders = {}
        
        i = 0
        previous_rank = 1
        for page in sorted_ranks:
            if sorted_ranks[page] < previous_rank:
                i += 1
            sorted_orders[page] = i
            previous_rank = sorted_ranks[page]

        return sorted_orders

    def pageInTopN(self, page: Page, n: int) -> bool:
        """
        Returns True if given page is in the top n of the network in terms of rank
        """
        sorted_orders = self.getRankingOrder()
        return sorted_orders[page] <= n
        
    def testUnconnected(self):
        """
        Prints if there are unconnected pages (pages with no link to or from them) in the network.
        """
        for page in self:
            if (len(page.incomingLinks)==0) & (len(page.outgoingLinks)==0):
                print("Page {} is unconnected".format(page.getName()))
        print("All unreported pages are connected to at least one other node via an incoming/outgoing link.")
        
    def getPage(self, name: str):
        """
        Returns page in this network with this name. 
        """
        for page in self:
            if (page.getName() == name):
                return page
        print("Error, no page with name " + name + " found!")
    
    # ----------------------------------------------------------
    # --------------------- VISUALIZATION ----------------------
    # ----------------------------------------------------------
    
    # ------------------- NODE-LINK DIAGRAMS ------------------- 
    def nxDiGraph(self):
        """Calculates networkx DiGraph"""
        # make a directed graph
        G = nx.DiGraph()
        
        # add all the edges and nodes to it
        for page in self:
            G.add_node(page)
            for other in page.outgoingLinks:
                G.add_edge(page, other)
        
        return G
    
    def plot1(self, names=False):
        """
        Plot using networkx.
        Node size based on *number of links*
        """
        G = self.nxDiGraph()
        
        # calculate a layout (force-directed)
        pos = nx.layout.spring_layout(G)

        # scale nodes by amount of links
        node_sizes = [1000 + 3000*len(page.outgoingLinks) for page in G.nodes()]
        
        if not names:
            # labels
            labels = {page:page.id for page in G.nodes()}
            
            nx.draw(G, pos, with_labels=True, labels=labels, node_size=node_sizes, arrowsize=40, font_color="white")
        else:
            nx.draw(G, pos, with_labels=True, node_size=node_sizes, arrowsize=40, font_color="white")
        plt.show()
    
    def plot2(self, names=False):
        """Plot using networkx. Node size based on *rank*"""
        G = self.nxDiGraph()
        
        # calculate a layout (force-directed)
        pos = nx.layout.spring_layout(G)

        # scale nodes by rank
        node_sizes = [1000 + 3000*page.rank for page in G.nodes()]
        
        if not names:
            # labels
            labels = {page:page.id for page in G.nodes()}
            
            nx.draw(G, pos, with_labels=True, labels=labels, node_size=node_sizes, arrowsize=40, font_color="white")
        else:
            nx.draw(G, pos, with_labels=True, node_size=node_sizes, arrowsize=40, font_color="white")
        plt.show()
        
    def plot0(self):
        """Plot using graphviz"""
        f = gv.Digraph()
        for page in self:
            f.node(str(page))
            for other in page.outgoingLinks:
                f.edge(str(page), str(other))

        return f
    
    # ----------------------- STATISTICS -----------------------
    def updateDf(self):
        """Update pandas DataFrame of this network."""
        
        # Prepare a dataframe
        df = pd.DataFrame(columns=['Name', 'Outdegree', 'Indegree'])
        
        df['Name'] = list(map(Page.getName, self.pages))
        df['Outdegree'] = list(map(Page.outDegree, self.pages))
        df.fillna(0, inplace=True)
        
        # Count indegree of every page
        for page in self:
            for link in page.outgoingLinks:
                df.at[link.id, 'Indegree'] += 1
        
        # Update dataframe
        self.df = df
        self.old_df = False
        
    def plot_outdegrees(self):
        """Plots a histogram of the outdegree of the pages in the network."""
        if self.old_df:
            self.updateDf()
        
        ax = sns.distplot(self.df['Outdegree'], kde=False, rug=True)
        ax.set_xlabel("Outdegree (number of links to other pages)")
        ax.set_ylabel("Frequency")
        
    def plot_indegrees(self):
        """Plots a histogram of the indegree of the pages in the network."""
        if self.old_df:
            self.updateDf()
        
        ax = sns.distplot(self.df['Indegree'], kde=False, rug=True)
        ax.set_xlabel("Indegree (number of links to this page from other pages)")
        ax.set_ylabel("Frequency")
        
    def describe(self):
        """Prints a table of summary statistics of this network."""
        if self.old_df:
            self.updateDf()
            
        return self.df.describe()
    
    def getDf(self):
        """Returns the dataframe of this network, updates it when needed."""
        if self.old_df:
            self.updateDf()
            
        return self.df
    
    def sinks(self):
        """Returns the sinks of the network."""
        if self.old_df:
            self.updateDf()
            
        return self.df[self.df['Outdegree'] == 0]
        
    def sources(self):
        """Returns the sources of the network."""
        if self.old_df:
            self.updateDf()
            
        return self.df[self.df['Indegree'] == 0]

### Example Wikipedia-YouTube-Twitter

In [ ]:
wiki = Page("Wikipedia")
yt = Page("YouTube")
twitter = Page("Twitter")

wiki.addLinkTo(yt)
wiki.addLinkTo(twitter)
yt.addLinkTo(twitter)
twitter.addLinkTo(yt)

In [ ]:
internet = Network()
internet.addPages(yt, wiki, twitter)

In [ ]:
pages = internet.getPages()
pages

In [ ]:
yt.incomingLinks

In [ ]:
yt.outgoingLinks

### Example California Network

In [ ]:
with open("california.txt") as f:
    content = f.read().splitlines()

In [ ]:
california = Network()
for line in content:
    if line[0] == 'n':
        california.addPage(Page(line.split()[2]))
    elif line[0] == 'e':
        n1 = california.pages[int(line.split()[1])]
        n2 = california.pages[int(line.split()[2])]
        
        n1.addLinkTo(n2)

## Ranking the pages

In [ ]:
class PageRanker:
    """A static class for ranking a network."""
    
    @staticmethod
    def rankCR(network: Network, steps: int = None, alpha: float = 0.85, epsilon: float = 1e-9, qis: List[float] = None): # rankCR = rankCertainRestart
        """
        Ranks a network via the PageRank algorithm.
        
        --- Parameters ---
        network: The Network to rank.
        steps: The number of iterations to do.
        alpha: The probability of going to another page via clicking a link.
        """
        initRanks = PageRanker.calculateInitialRanking(network)
        
        if qis == None:
            PageRanker.calculateJumpingProbabilities(network)
        else:
            for i, page in enumerate(network):
                page.q = qis[i]
        
        if steps != None:
            for _ in range(0, steps):
                PageRanker.stepRankCR(network, alpha)
        
        else:
            n = 0
            oldRanks = initRanks
            while True:
                n += 1
                newRanks = PageRanker.stepRankCR(network, alpha)
#                 print(newRanks)
                d = max([abs(newRanks[page] - oldRanks[page]) for page in network])

                if d < epsilon:
                    break
                
                oldRanks = newRanks
                    
#             print(f'Iterations: {n}')
            return n
    
    @staticmethod
    def stepRankCR(network: Network, alpha: float):
        """
        Does one step of the random-restart ranking
        """
        newRanks = {}
        for page in network:
            newRanks[page] = 0
            # Sum dot product of column and stationary distribution
            for other in network:
                if other.outDegree() > 0:
                    newRanks[page] += other.rank * page.q * (1 - alpha)
                else:
                    newRanks[page] += other.rank * page.q
                if other.linksTo(page):
                    newRanks[page] += other.rank * alpha / other.outDegree()

        for page in network:
            page.rank = newRanks[page]
            
        return newRanks
    
    @staticmethod
    def rank(network: Network, steps: int = None, alpha: float = 0.85, epsilon: float = 1e-9, qis: List[float] = None, ris: List[float] = None, debug=False):
        """
        Ranks a network via the PageRank algorithm.
        Runtime O(steps * #pages * max. incoming links for a page)
        
        --- Parameters ---
        network: The Network to rank.
        steps: The number of iterations to do.
        alpha: The probability of going to another page via clicking a link.
        """
        
        if qis == None:
            PageRanker.calculateJumpingProbabilities(network)
        else:
            for i, page in enumerate(network):
                page.q = qis[i]
        
        initRanks = {}
        if ris == None:
            initRanks = PageRanker.calculateInitialRanking(network)
        else:
            for i, page in enumerate(network):
                page.rank = ris[i]
                initRanks[page] = ris[i]
        
        if steps != None:
            for _ in range(0, steps):
                PageRanker.stepRank(network, alpha)
                rankSum = sum([page.rank for page in network])
                for page in network:
                    page.rank = page.rank/rankSum
                    
                return
            
        else:
            n = 0
            oldRanks = initRanks
            if debug:
                print(initRanks)
            
            while True:
                n += 1
                newRanks = PageRanker.stepRank(network, alpha)
                if debug:
                    print(newRanks)
                d = max([abs(newRanks[page] - oldRanks[page]) for page in network])

                if d < epsilon:
                    break
                
                oldRanks = newRanks
                    
            rankSum = sum([page.rank for page in network])
            for page in network:
                page.rank = page.rank/rankSum

#             print(f'Iterations: {n}')
            return n

        
    @staticmethod
    def stepRank(network: Network, alpha: float):
        """
        Does one step of the normalized ranking
        """
        newRanks = {}
        for page in network:
            newRanks[page] = 0
            for other in page.incomingLinks:
                newRanks[page] += other.rank / other.outDegree()
            newRanks[page] *= alpha
            newRanks[page] += (1-alpha) * page.q

        for page in network:
            page.rank = newRanks[page]
            
        return newRanks
    
    @staticmethod
    def calculateInitialRanking(network: Network):
        """
        Calculates an initial ranking for the given network.
        Currently just gives each page a ranking of 1 / size of the network.
        """
        for page in network:
            page.rank = 1.0 / network.size()
#             print("initial rank for " + str(network.size()) + " pages is " + str((1.0 / network.size())) )
            # When looking at the above print command for large networks, the sum doesn't necessarily add to 1 perfectly
            # anymore due to rounding errors
        
        return {page:1.0/network.size() for page in network}
            
    @staticmethod
    def calculateJumpingProbabilities(network: Network):
        """Calculates the probabilities of jumping to each page."""
        for page in network:
            page.q = 1.0 / network.size()


### Example Wikipedia-YouTube-Twitter Calculation

In [ ]:
PageRanker.calculateInitialRanking(internet)

#### Wikipedia-YouTube-Twitter with the Certain Restart Version

In [ ]:
PageRanker.rankCR(internet)
internet.showRanking()
internet.debugRanking()

As long as the network does not have a sink, the ranks seem to be calculated correctly and still sum to 1. 

#### Wikipedia-YouTube-Twitter with Normalizer Version

In [ ]:
PageRanker.rank(internet) 

In [ ]:
internet.showRanking()

In [ ]:
internet.debugRanking() # This one is the one with the normalizer feature, naturally it sums perfectly to 1 for this simple network

### Example California Network Calculation

#### Certain Restart Version

In [ ]:
# slow (this is only 1 iteration!)
# PageRanker.rankCR(california, 1, 0.5)
# california.showRanking()
# california.debugRanking()

#### Normalizer Version

In [ ]:
PageRanker.rank(california)
california.showRanking()
california.debugRanking()

### Edge Case: Source -> Sink

#### Normalizer Version

In [ ]:
criminal = Network()
page1 = Page("Source")
page2 = Page("Sink")
page1.addLinkTo(page2)
criminal.addPages(page1, page2)

In [ ]:
#criminal.plot0()

In [ ]:
PageRanker.rank(criminal, 10, 0.5)
criminal.showRanking()
criminal.debugRanking()

In [ ]:
nx.pagerank(criminal.nxDiGraph(), alpha=0.5, max_iter=10)

#### Certain Restart Version

In [ ]:
PageRanker.rankCR(criminal, 10, 0.5)
criminal.showRanking()
criminal.debugRanking()

### Timo's handsolved network

In [ ]:
handsolved = Network()
w1 = Page("Website 1")
w2 = Page("Website 2")
w3 = Page("Website 3")
w4 = Page("Website 4")
w5 = Page("Website 5")

w1.addLinksTo(w2, w3, w4, w5)
w2.addLinksTo(w3, w4, w5)
w3.addLinksTo(w1, w2, w4, w5)
w4.addLinksTo(w2, w3)
w5.addLinkTo(w4)

handsolved.addPages(w1, w2, w3, w4, w5)
#handsolved.plot0()

In [ ]:
PageRanker.rankCR(handsolved, 100, 0.5)
handsolved.showRanking()

In [ ]:
print(f'Website 1: {36/283}')
print(f'Website 2: {297/1415}')
print(f'Website 3: {308/1415}')
print(f'Website 4: {378/1415}')
print(f'Website 5: {252/1415}')

## Visualization of (small) networks

### With networkx

In [ ]:
# size based on links
internet.plot1()

In [ ]:
# plot with names (works in this case, but often labels are too large for the nodes)
internet.plot1(names=True)

In [ ]:
#size based on rank
internet.plot2()

### With graphviz

In [ ]:
internet.plot0()

### Homepage Network

In [ ]:
def makeHomepageNetwork(sinkSiteAmount: int = 100):
    homepageNetwork= Network()
    homepage = Page("Homepage")
    for i in range(0,sinkSiteAmount):
        p = Page("test" + str(i))
        homepage.addLinkTo(p)
        homepageNetwork.addPage(p)
    homepageNetwork.addPage(homepage)
    return homepageNetwork

### Endpage Network

In [ ]:
# Configurable parameter
def makeEndpageNetwork(sourceSiteAmount: int = 100):
    endpageNetwork = Network()
    endpage = Page("Endpage")
    for i in range(0,sourceSiteAmount):
        p = Page("test" + str(i))
        p.addLinkTo(endpage)
        endpageNetwork.addPage(p)
    endpageNetwork.addPage(endpage)
    return endpageNetwork

## Statistics

In [ ]:
internet.sources()

In [ ]:
internet.sinks()

In [ ]:
california.sinks()

In [ ]:
california.sources()

In [ ]:
internet.plot_outdegrees()

In [ ]:
internet.plot_indegrees()

In [ ]:
internet.describe()

In [ ]:
california.plot_outdegrees()

In [ ]:
california.plot_indegrees()

In [ ]:
california.describe()

# More Network Generators

In [ ]:
def makeClusterNetwork(clusterAmount: int=3, clusterSize: int=10, forwardLinkClusters: bool=True, backwardLinkClusters: bool=True):
    """
    --- Parameters ---
    clusterAmount: Amount of clusters in the network.
    clusterSize: Amount of pages per cluster.
    forwardLinkClusters: Should there exist a link from cluster i to i+1 for all i?
    backwardLinkClusters: Should there exist a link from cluster i to i-1 for all i?
    """
    currentCluster = []
    clusterNetwork = Network()
    for i in range(0, clusterAmount):
        for j in range(0, clusterSize):
            p = Page("cluster_" + str(i) + "_site_" + str(j))
            clusterNetwork.addPage(p)
            for otherPage in currentCluster:
                p.addLinkTo(otherPage)
                otherPage.addLinkTo(p)
            currentCluster.append(p)    
        currentCluster = []
    
    if forwardLinkClusters:
        for i in range(0, clusterAmount):
            if (i==clusterAmount - 1):
                nextClusterID = 0
            else:
                nextClusterID = i + 1
            clusterNetwork.getPage("cluster_" + str(i) + "_site_0").addLinkTo(clusterNetwork.getPage("cluster_" + str(nextClusterID) + "_site_0"))

    if backwardLinkClusters:
        for i in range(0, clusterAmount):
            if (i==0):
                previousClusterID = clusterAmount - 1
            else:
                previousClusterID = i - 1
            clusterNetwork.getPage("cluster_" + str(i) + "_site_0").addLinkTo(clusterNetwork.getPage("cluster_" + str(previousClusterID) + "_site_0"))

    return clusterNetwork

In [ ]:
# Fully linked network where every site has a link to every other site except for the last sinkAmount (parameter) sites.
# The last site is a sink and only has incoming links.
def makeNetworkSinks(sinkAmount: int = 3, networkSize: int = 10):
    """
    --- Parameters ---
    sinkAmount: Amount of pages in the network without outgoing links.
    networkSize: Amount of pages in the network
    Beware, the function requires sinkAmount <= networkSize!
    """
    networkSinks = Network()
    for i in range(0, networkSize):
        p = Page("site_" + str(i))
        networkSinks.addPage(p)
    for page in networkSinks:
        pageID = int(page.getName()[5:])
        # if not one of the sinks
        if pageID >= sinkAmount:
            # then create links to all other pages which are not itself
            for otherPage in networkSinks:
                otherPageID = int(otherPage.getName()[5:])
                if not(otherPageID == pageID):
                    page.addLinkTo(otherPage)
    return networkSinks

In [ ]:
# Every site points to the next in a big "circle"

# Configurable Parameter
circleSize = 50
# Only forward-pointing links or also backward-pointing links?
twoWay = True
def makeNetworkCircle(circleSize: int = 50, twoWay: bool = True):
    """
    --- Parameters ---
    circleSize: Amount of pages in the circular network.
    twoWay: Boolean representing if there are only forward-pointing links or also backward-pointing links,
    so if the "circle" is traversable in two directions or in 1 direction.
    """
    networkCircle = Network()
    for i in range(0, circleSize):
        p = Page("site_" + str(i))
        networkCircle.addPage(p)
    for page in networkCircle:
        pageID = int(page.getName()[5:])
        if not(pageID == circleSize - 1):
            nextPageID = pageID + 1
        else:
            nextPageID = 0
        page.addLinkTo(networkCircle.getPage("site_" + str(nextPageID)))
        if twoWay:
            if not(pageID == 0):
                previousPageID = pageID - 1
            else:
                previousPageID = circleSize - 1
            page.addLinkTo(networkCircle.getPage("site_" + str(previousPageID)))
    return networkCircle


# The Random "Spatial" Network

This random network of websites is supposed to reflect the spatial nature of the internet. For example, Chinese websites will probably have a lot of links to other Chinese websites and similarly for for example Spanish websites. However, between the two subnetworks (or clusters), there will probably be a lot fewer links. This example network is supposed to reflect that.

In [ ]:
class SpatialPage(Page):
    def __init__(self, name, x, y):
        super().__init__(name)
        self.x = x
        self.y = y
        
    def squaredDistanceTo(self, other):
        return (self.x-other.x)^2 + (self.y-other.y)^2

In [ ]:
def makeSpatialNetwork(n, xmax, ymax, r):
    """
    Returns a spatial network
    === Parameters ===
    n: amount of pages
    xmax: largest x coordinate
    ymax: largest y coordinate
    r: range which determines whether cells are neighbours
    """
    spatialNetwork = Network()
    
    for i in range(n):
        x = round(xmax * random.random())
        y = round(ymax * random.random())
        
        p = SpatialPage(f'Page {i}', x, y)
        spatialNetwork.addPage(p)
    
    for page in spatialNetwork:
        for other in spatialNetwork:
            if page == other:
                continue
            
            d = page.squaredDistanceTo(other)
            if d <= r^2:
                # maybe do this with a link probability?
                page.addLinkTo(other)
    
    return spatialNetwork

In [ ]:
test = makeSpatialNetwork(5, 10, 10, 2)
test.plot0()

In [ ]:
# Configurable parameters
siteAmount = 100
# Coordinate range
xmax = 10
ymax = 10
# "Closeness" bound. If sites are within this range of each other, they will link to each other.
closeRange = 5

# Constants
spatialNetwork= Network()
df_coords = pd.DataFrame(columns = ["x", "y"])
adjust = max(xmax,ymax) // 10 + 1
closeRangeSqr = closeRange*closeRange

for i in range(0,siteAmount):
    # Determine random integer coordinates between 0 and 100 for all sites
    x = round(xmax * random.random())
    y = round(ymax * random.random())
    
    
    df_coords.loc[i] = [x, y]
    # Keep this information in the "name" string of the website. 
    # Add the filler character "f" such that the coordinates fill up 3 characters

    p = Page("{}".format(x).rjust(adjust, "f") + "_" + "{}".format(y).rjust(adjust, "f"))
    spatialNetwork.addPage(p)
    #print("Adding site " + p.getName())
    
    for other in spatialNetwork:
        # Take slices of the other's name and strip zeros accordingly to het back the coordinates
        otherName = other.getName()
        otherX = otherName[:adjust].lstrip("f")
        otherY = otherName[adjust+1:].lstrip("f")
        #print("Other site found with X = " + otherX + " and Y = " + otherY)
        
        # If the other site and site p are close enough (and other =/= p !!)
        distance = (int(otherX)-x)*(int(otherX)-x) + (int(otherY)-y)*(int(otherY)-y)
        if (distance <= closeRangeSqr)&(0<distance):
            other.addLinkTo(p)
            p.addLinkTo(other)

In [ ]:
PageRanker.rankCR(spatialNetwork, 1000, 0.5)
spatialNetwork.showRanking()
spatialNetwork.debugRanking()

In [ ]:
df_coords.plot(kind="scatter", x="x", y="y");

In [ ]:
# spatialNetwork.plot2()

In [ ]:
spatialNetwork.testUnconnected()

In [ ]:
PageRanker.rank(spatialNetwork)

## Completely Random Network

In [ ]:
def makeRandomNetwork(n: int = 100, successProbability: float = 0.1):
    randomNetwork= Network()

    for i in range(0,n):
        p = Page("Page {}".format(i))
        randomNetwork.addPage(p)

    for p in randomNetwork:
        for other in randomNetwork:
            # Check if equal
            if not(other.getName() == p.getName()):
                if (random.random() < successProbability):
                    other.addLinkTo(p)
                    
    return randomNetwork

## 2WB20 Canvas Page

In [ ]:
canvas = Network()
w1 = Page("Canvas Main Page")
w2 = Page("Modules")
w3 = Page("Schedule")
w4 = Page("Syllabus")
w5 = Page("Discussions")
w6 = Page("Video Lectures")
w7 = Page("Lecture Notes")
w8 = Page("Instruction Exercises")
w9 = Page("Modelling")
w10 = Page("Handouts")

w1.addLinksTo(w2, w3, w4, w5, w6)
w2.addLinksTo(w1, w6, w7, w8, w9, w10)
w3.addLinksTo(w1, w8, w10)
w4.addLinksTo(w1, w3)
w5.addLinkTo(w1)
w6.addLinkTo(w2)
w7.addLinkTo(w2)
w8.addLinkTo(w2)
w9.addLinkTo(w2)
w10.addLinkTo(w2)

canvas.addPages(w1, w2, w3, w4, w5, w6, w7, w8, w9, w10)
canvas.plot0()

In [ ]:
PageRanker.rank(canvas)
canvas.showRanking()

In [ ]:
personalization = [0.7, 0.3/9, 0.3/9, 0.3/9, 0.3/9, 0.3/9, 0.3/9, 0.3/9, 0.3/9, 0.3/9]

In [ ]:
PageRanker.rank(canvas, qis=personalization)
canvas.showRanking()

In [ ]:
canvas.getRankingOrder()

In [ ]:
# Canvas main page in top 5
canvas.pageInTopN(w1, 5)

In [ ]:
# Handouts page not in top 5
canvas.pageInTopN(w10, 5)

## Testing Area

In [ ]:
list_of_results = []
for i in range(0,10):
    net = makeEndpageNetwork((i+1) * 1000)
    list_of_results.append(PageRanker.rank(net))
print(list_of_results)


In [ ]:
network = makeHomepageNetwork(1000)
PageRanker.rank(network, 1000, 0.5)
network.showRanking()

In [ ]:
1/1001

In [ ]:
n = 100
rs = [1] + (n-1)*[0]

In [ ]:
circle = makeNetworkCircle(n, True)

In [ ]:
# circle.plot0()

In [ ]:
PageRanker.rank(circle, ris=rs, alpha=0.85)

In [ ]:
PageRanker.rank(makeRandomNetwork(100, 0.015))

In [ ]:
PageRanker.rank(makeRandomNetwork(1000, 0.1))

In [ ]:
iterations = {}
n = 100
# n=1
nNodes = 500
for i in range(n):
    p = i/n
    net = makeRandomNetwork(nNodes, p)
    iterations[p] = PageRanker.rank(net)

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d
x = list(iterations.keys())
y = list(iterations.values())

fig, ax = plt.subplots(1, 1)
# ax.plot(x, y)
ax.plot(x, gaussian_filter1d(y, sigma=2))
ax.set_xlabel("Link probability")
ax.set_ylabel("# of iterations")
ax.set_title(f'PR convergence in random graph, {nNodes} nodes');

In [ ]:
PageRanker.rank(makeHomepageNetwork(3), alpha=0.99, debug=True)

In [ ]:
# ax.get_figure().savefig("plot.pdf")